<a href="https://colab.research.google.com/github/mrreyesm/SECOM/blob/main/SECOM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 

In [2]:
url="https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom.data"
secom_data = pd.read_csv(url, sep = ' ')

In [3]:
secom_data.columns = ["FEATURE_"+str(col+1) for col in range(0,len(secom_data.columns))]

In [4]:
secom_data.head()

,FEATURE_1,FEATURE_2,FEATURE_3,FEATURE_4,FEATURE_5,FEATURE_6,FEATURE_7,FEATURE_8,FEATURE_9,FEATURE_10,...,FEATURE_581,FEATURE_582,FEATURE_583,FEATURE_584,FEATURE_585,FEATURE_586,FEATURE_587,FEATURE_588,FEATURE_589,FEATURE_590
0,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,-0.0005,...,0.0060,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045
1,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,0.0041,...,0.0148,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602
2,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,-0.0124,...,0.0044,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432
3,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,-0.0031,...,NaN,NaN,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432
4,2946.25,2432.84,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5287,0.0167,...,0.0052,44.0077,0.4949,0.0189,0.0044,3.8276,0.0342,0.0151,0.0052,44.0077


In [5]:
url="https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom_labels.data"
secom_labels = pd.read_csv(url, sep = " ")

In [6]:
secom_labels.columns = ("STATUS", "TIMESTAMP")

In [8]:
secom_labels.head()

,STATUS,TIMESTAMP
0,-1,19/07/2008 12:32:00
1,1,19/07/2008 13:17:00
2,-1,19/07/2008 14:43:00
3,-1,19/07/2008 15:22:00
4,-1,19/07/2008 17:53:00
